In [1]:
from ANEMO import ANEMO
from ANEMO.edfreader import read_edf
import pickle
import numpy as np

In [2]:
sujets = ['AM','BMC','CS','DC','FM','IP','LB','OP','RS','SR','TN','YK']
times = ['2017-10-23_100057','2017-09-26_095637', '2017-10-03_134421','2017-09-27_161040',
        '2017-10-03_143803','2017-09-28_115250', '2017-09-20_151043','2017-10-26_121823',
        '2017-11-08_094717','2017-11-16_153313', '2017-11-08_150410','2017-11-17_172706']

In [3]:
def Fit_trial(trial=22, block=0, s=0,
              time_sup=280, step_fit=2, do_whitening=False,
              before_sacc=5, after_sacc=15) :


    with open('data/enregistrement_%s_%s.pkl'%(sujets[s], times[s]), 'rb') as fichier :
        exp = pickle.load(fichier, encoding='latin1')
    data = read_edf('data/enregistrement_%s_%s.asc'%(sujets[s], times[s]), 'TRIALID')

    Fit = ANEMO.Fit(exp)
    A = ANEMO(exp)
    
    
    from lmfit import  Model, Parameters

    
    #-----------------------------------------------------------------------------
    data_trial_ = data[trial+200*block]
    arg = A.arg(data_trial_, trial=trial, block=block)
    
    data_trial = A.velocity_NAN(**arg)
    
    trackertime = arg.trackertime
    TargetOn, StimulusOf    = arg.TargetOn, arg.StimulusOf
    saccades    = arg.saccades

    trackertime = arg.trackertime
    t_0 = trackertime[0]

    dir_target = arg.dir_target
    #-----------------------------------------------------------------------------

    if   step_fit == 1 : vary = True
    elif step_fit == 2 : vary = False


    if time_sup is not None :
        data_trial = data_trial[:-time_sup]
        trackertime = trackertime[:-time_sup]

    if do_whitening :
        for x in range(len(data_trial)) :
            if np.isnan(data_trial[x]) :
                if x == 0 : data_trial[x] = 0
                else :      data_trial[x] = data_trial[x-1]

        data_trial = whitening(data_trial)

    
    value_latency, value_steady_state, value_anti = A.classical_method.Full(data_trial, TargetOn-t_0)
        
    

    #----------------------------------------------
    max_latency = []
    for s in range(len(saccades)) :
        if (saccades[s][0]-t_0) >= (TargetOn-t_0+100) : max_latency.append((saccades[s][0]-t_0))
    if max_latency == [] :                              max_latency.append(len(trackertime))
    max_latency = max_latency[0]

    if value_latency >= max_latency-50 : value_latency = max_latency-150
    if value_latency > 250 :             value_latency = TargetOn-t_0+100
    #----------------------------------------------

    param_fit=[{'name':'steady_state', 'value':value_steady_state, 'min':5.,                 'max':40.,             'vary':True  },
               {'name':'dir_target',   'value':dir_target,         'min':None,               'max':None,            'vary':False },
               {'name':'a_anti',       'value':value_anti,         'min':-40.,               'max':40.,             'vary':True  },
               {'name':'latency',      'value':value_latency,      'min':TargetOn-t_0+75,    'max':max_latency,     'vary':True  },
               {'name':'start_anti',   'value':TargetOn-t_0-100,   'min':StimulusOf-t_0-200, 'max':TargetOn-t_0+75, 'vary':'vary'},
               {'name':'tau',          'value':15.,                'min':13.,                'max':80.,             'vary':'vary'}]

    inde_vars = {'x':np.arange(len(trackertime))}

    equation = A.Equation.fct_velocity

    params = Parameters()
    model = Model(equation, independent_vars=inde_vars.keys())

    for num_par in range(len(param_fit)) :

        if 'expr' in param_fit[num_par].keys() :
            params.add(param_fit[num_par]['name'], expr=param_fit[num_par]['expr'])
        else :
            if param_fit[num_par]['vary'] == 'vary' : var = vary
            else :                                    var = param_fit[num_par]['vary']
            params.add(param_fit[num_par]['name'],
                       value=param_fit[num_par]['value'],
                       min=param_fit[num_par]['min'],
                       max=param_fit[num_par]['max'],
                       vary=var)

    params.add('do_whitening', value=do_whitening, vary=False)

    if step_fit == 1 :

        result_deg = model.fit(data_trial, params, nan_policy='omit', **inde_vars)

    elif step_fit == 2 :

        out = model.fit(data_trial, params, nan_policy='omit', **inde_vars)

        # make the other parameters vary now
        for num_par in range(len(param_fit)) :
            if 'vary' in param_fit[num_par].keys() :
                if param_fit[num_par]['vary'] == 'vary' :
                    out.params[param_fit[num_par]['name']].set(vary=True)

        result_deg = model.fit(data_trial, out.params, method='nelder', nan_policy='omit', **inde_vars)

    return result_deg

In [4]:
lala = Fit_trial()

In [5]:
lala

In [6]:
print(lala.ci_report())

/usr/local/lib/python3.6/dist-packages/lmfit/confidence.py:318: UserWarning: rel_change=0.0 < 1e-05 at iteration 2 and prob(steady_state=4.9996933475860335) = 0.005363616297350077 < max(sigmas).
  warn(errmsg)
/usr/local/lib/python3.6/dist-packages/lmfit/confidence.py:310: UserWarning: maxiter=200 reached and prob(steady_state=5.030825672694556) = 0.46280670155752085 < max(sigmas).
  warn(errmsg)
/usr/local/lib/python3.6/dist-packages/lmfit/confidence.py:318: UserWarning: rel_change=-1.0 < 1e-05 at iteration 4 and prob(latency=1319.09130035226) = 0.0 < max(sigmas).
  warn(errmsg)
/usr/local/lib/python3.6/dist-packages/lmfit/confidence.py:318: UserWarning: rel_change=0.0 < 1e-05 at iteration 4 and prob(tau=4.049162978018495) = 0.9884404041459219 < max(sigmas).
  warn(errmsg)


                 99.73%    95.45%    68.27%    _BEST_    68.27%    95.45%    99.73%
 steady_state:      -inf      -inf      -inf   5.00000      +inf      +inf      +inf
 a_anti      :  -3.96047  -2.63993  -1.32163   5.31395  +1.31001  +2.75519  +3.94885
 latency     : -21.81416 -16.69852  -4.451641300.07397      +inf      +inf      +inf
 start_anti  :-138.33275 -94.55131 -49.06810 975.99966 +42.57387 +94.26798+175.50848
 tau         :      -inf  -9.88653  -5.06434  25.44139  +4.86154 +10.13943 +15.73523


In [7]:
for a in lala.params :
    b = lala.params[a]
    print(a, (b.stderr/b.value)*100)

steady_state 0.0030672241056896774
dir_target -0.0
a_anti 45.78307780557688
latency 0.36569716737371893
start_anti 8.463612378393579
tau 21.021087523211463


ZeroDivisionError: division by zero